In [1]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

In [2]:
import tensorflow as tf
import cv2
import numpy as np
import os
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the trained model
model = tf.keras.models.load_model('facial_expression_model1.h5')

# Define the classes (adjust based on your dataset)
classes = ['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture from the camera
cap = cv2.VideoCapture(0)

# Prediction frame skip counter
frame_counter = 0
frame_skip = 5

# Directory to store user images
user_images_dir = 'user_images'
os.makedirs(user_images_dir, exist_ok=True)

# Variable to hold the last predicted label and recognized name
last_predicted_label = None
recognized_name = None

# Function to capture and save user image
def capture_user_image(user_name):
    ret, frame = cap.read()
    if ret:
        user_image_path = os.path.join(user_images_dir, f"{user_name}.jpg")
        cv2.imwrite(user_image_path, frame)
        print(f"Image saved for user: {user_name}")
    return user_image_path

# Load user data
user_data = {}
if os.path.exists('user_data.npy'):
    user_data = np.load('user_data.npy', allow_pickle=True).item()

# Function to show a dialog box for user name input
def input_user_name():
    name = ''
    while True:
        cv2.putText(frame, "Enter Name: " + name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow('Live Camera Feed', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Escape key to cancel
            return None
        elif key == 13:  # Enter key to confirm
            return name
        elif len(name) < 20 and (key >= 32 and key <= 126):  # ASCII range for printable characters
            name += chr(key)
        elif key == 8:  # Backspace
            name = name[:-1]

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    # Process each detected face
    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract the face region for prediction
        face_roi = frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (224, 224))
        normalized_face = preprocess_input(resized_face)
        input_face = np.expand_dims(normalized_face, axis=0)

        # Predict every frame_skip frames
        if frame_counter % frame_skip == 0:
            predictions = model.predict(input_face)
            predicted_class = np.argmax(predictions[0])
            predicted_label = classes[predicted_class]

            # Check for user recognition
            recognized_name = None
            for user_name, user_image_path in user_data.items():
                user_image = cv2.imread(user_image_path)
                user_resized = cv2.resize(user_image, (224, 224))
                user_normalized = preprocess_input(user_resized)
                user_input = np.expand_dims(user_normalized, axis=0)
                
                # Compare the embeddings (optional: replace with a proper distance measure)
                if np.array_equal(np.argmax(model.predict(user_input)[0]), np.argmax(predictions[0])):
                    recognized_name = user_name
                    break

            # Update the last predicted label
            if recognized_name:
                last_predicted_label = recognized_name
            else:
                last_predicted_label = predicted_label

        # Display the recognized name and expression inside the rectangle
        if recognized_name:
            name_color = (0, 255, 0)  # Green color for the name
            cv2.putText(frame, recognized_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, name_color, 2)

        # Display the predicted label in white
        cv2.putText(frame, predicted_label, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Display the resulting frame
    cv2.imshow('Live Camera Feed', frame)

    # Increment frame counter
    frame_counter += 1

    # Check for key press to add a new user
    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):  # Press 'a' to add a new user
        user_name = input_user_name()
        if user_name:
            user_image_path = capture_user_image(user_name)
            user_data[user_name] = user_image_path
            np.save('user_data.npy', user_data)

    # Break the loop on 'q' key press
    if key == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


In [3]:
pip install pyinstaller


   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.3 MB 1.5 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.3 MB 1.5 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.3 MB 1.5 MB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 610.3 kB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 610.3 kB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 610.3 kB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 610.3 kB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 610.3 kB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.3 MB 610.3 kB/s eta 0:00:01
   ----------------------- -

In [1]:
pyinstaller --onefile my.ipynb

SyntaxError: invalid syntax (2301843362.py, line 1)